# 🛸 UFO Analysis & Visualization

**Goal**: Query our engineered data using **Amazon Athena** and visualize trends.

### Prerequisites:
1.  **Glue Crawler** must have finished running.
2.  Table **`ufo_analytics`** must exist in database **`ufo_db`**.
3.  We use `boto3` to run SQL queries against Athena.

In [ ]:
import boto3
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Setup Athena Client
session = boto3.Session()
athena = session.client("athena", region_name="us-east-1")

# CONFIG
DATABASE = "ufo_db"
TABLE = "ufo_analytics"
S3_OUTPUT = "s3://egirgis-datalake-v1/athena_results/"  # Where Athena stores query results

### Helper Function: Run Athena Query
Athena queries are asynchronous. We need to submit columns and wait for the result.

In [ ]:
def run_query(query, database, s3_output):
    # 1. Start Query
    response = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database},
        ResultConfiguration={'OutputLocation': s3_output}
    )
    query_execution_id = response['QueryExecutionId']
    
    # 2. Wait for Completion
    print(f"Running Query (ID: {query_execution_id})...", end="")
    while True:
        stats = athena.get_query_execution(QueryExecutionId=query_execution_id)
        status = stats['QueryExecution']['Status']['State']
        if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break
        print(".", end="")
        time.sleep(1)
    
    print(f"\nStatus: {status}")
    if status == 'FAILED':
        print(stats['QueryExecution']['Status']['StateChangeReason'])
        return None
    
    # 3. Get Results using Pandas (reading the CSV output from S3)
    # Construct the S3 path of the result file
    result_s3_path = f"{s3_output}{query_execution_id}.csv"
    return pd.read_csv(result_s3_path)


### 1. Simple Data Check

In [ ]:
sql = f"SELECT * FROM {TABLE} LIMIT 10;"
df_sample = run_query(sql, DATABASE, S3_OUTPUT)
df_sample.head()

### 2. Analysis: Sightings Per Year
Has alien activity increased over time? 👽

In [ ]:
sql_trend = f"""
    SELECT year, COUNT(*) as count 
    FROM {TABLE} 
    WHERE year IS NOT NULL
    GROUP BY year 
    ORDER BY year;
"""
df_trend = run_query(sql_trend, DATABASE, S3_OUTPUT)

# Check types
df_trend.head()

### 3. Visualization

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_trend['year'], df_trend['count'], marker='o', linestyle='-', color='lime')
plt.title('UFO Sightings Over Time', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Sightings', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

### 4. Analysis: Most Common Shapes

In [ ]:
sql_shapes = f"""
    SELECT shape, COUNT(*) as frequency 
    FROM {TABLE} 
    GROUP BY shape 
    ORDER BY frequency DESC 
    LIMIT 10;
"""
df_shapes = run_query(sql_shapes, DATABASE, S3_OUTPUT)

# Fix for matplotlib error: ensure data is string and handle NaNs
df_shapes['shape'] = df_shapes['shape'].fillna("Unknown").astype(str)

plt.figure(figsize=(10, 6))
plt.barh(df_shapes['shape'], df_shapes['frequency'], color='purple')
plt.xlabel('Frequency')
plt.title('Top 10 UFO Shapes Reported')
plt.gca().invert_yaxis()  # Highest at top
plt.show()